In [6]:
import sys
import boto3
import json
import pandas as pd
from my_secrets import my_secrets

In [7]:
###########################################
#                READ DATA                #
###########################################

raw_data = pd.read_csv('data/Corona_NLP_test.csv').tail(10)
mturk_data = pd.read_csv('./assignments_1-25.csv')
mturk_data['ATime'] = pd.to_datetime(mturk_data['AcceptTime'])
mturk_data['STime'] = pd.to_datetime(mturk_data['SubmitTime'])

f = open('results_1-25.json')
results = json.load(f)
res_df = pd.DataFrame.from_records(results)


In [17]:
# for res in results:
#     print(res['hit_id'] + " " + str(res['sentiment']))

res_dict = {
    "389A2A3052C7KXMI1CS5Z71AN5BC0I": 3,
    "3AJA9FLWTQSPU4N8W47CDHM96CKFIR": 5,
    "3X7837UUBRSP6ME3DTP8VCOWQS66JR": 4,
    "3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3": 1,
    "3O0M2G5VDKWMLNXG8T5G9KQFYWO94E": 2,
    "37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L": 4,
    "3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3": 2,
    "3TFJJUELTVJEG7PWGZWJIVUPK3VC22": 3,
    "3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2": 1,
    "3TL87MO8D0JRK5VEH74P5XE1NGEFL5": 5,
}

res_df['HITId'] = res_df['hit_id'] 

In [18]:
mturk_data['GroundTruth'] = mturk_data.apply(lambda x: res_dict[x['HITId']], axis=1)
mturk_data['MinDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(minutes=1)
mturk_data['SecDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(seconds=1)

mturk_data.head()

,AssignmentId,WorkerId,HITId,AutoApprovalTime,AcceptTime,SubmitTime,Answer,ATime,STime,GroundTruth,MinDiff,SecDiff
0,3RYC5T2D7IOG00DEQW2SSNWJMLUPRE,A2IXK982JL0ECO,389A2A3052C7KXMI1CS5Z71AN5BC0I,2023-02-24 14:21:49-05:00,2023-01-25 14:21:01-05:00,2023-01-25 14:21:49-05:00,1,2023-01-25 14:21:01-05:00,2023-01-25 14:21:49-05:00,3,0.800000,48.0
1,3SLE99ER028NYXYVH8E4J6TTEYYZBW,A2JX2VIJWXHS4X,389A2A3052C7KXMI1CS5Z71AN5BC0I,2023-02-24 14:24:33-05:00,2023-01-25 14:24:13-05:00,2023-01-25 14:24:33-05:00,4,2023-01-25 14:24:13-05:00,2023-01-25 14:24:33-05:00,3,0.333333,20.0
2,3QAPZX2QNJ8W8ZS9UL81CEN32L3206,A2JX2VIJWXHS4X,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,2023-02-24 14:26:06-05:00,2023-01-25 14:25:56-05:00,2023-01-25 14:26:06-05:00,4,2023-01-25 14:25:56-05:00,2023-01-25 14:26:06-05:00,5,0.166667,10.0
3,3JJVG1YBEQSPRJE6AK7OU2MYDZLB5W,A2CRQ0XPXN1P0K,3X7837UUBRSP6ME3DTP8VCOWQS66JR,2023-02-23 22:02:17-05:00,2023-01-24 22:02:12-05:00,2023-01-24 22:02:17-05:00,1,2023-01-24 22:02:12-05:00,2023-01-24 22:02:17-05:00,4,0.083333,5.0
4,3HMVI3QICYNMG96YJPJWZN7IMS41YP,A2JX2VIJWXHS4X,3X7837UUBRSP6ME3DTP8VCOWQS66JR,2023-02-24 14:25:38-05:00,2023-01-25 14:25:31-05:00,2023-01-25 14:25:38-05:00,4,2023-01-25 14:25:31-05:00,2023-01-25 14:25:38-05:00,4,0.116667,7.0


In [19]:
# Which tasks took the longest?
# Which workers worked the fastest?

Based on the average of mturk responses, how accurate are the existing labels?

In [23]:
avg_df = mturk_data[['HITId', 'Answer', 'GroundTruth']]
avg_df.head(3)


,HITId,Answer,GroundTruth
0,389A2A3052C7KXMI1CS5Z71AN5BC0I,1,3
1,389A2A3052C7KXMI1CS5Z71AN5BC0I,4,3
2,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,4,5


In [35]:
agg_df = avg_df.groupby("HITId")["Answer", "GroundTruth"].agg(["mean", "std", "max", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')


C:\Users\pherk\AppData\Local\Temp\ipykernel_16740\8824565.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("HITId")["Answer", "GroundTruth"].agg(["mean", "std", "max", "count"])


In [40]:
agg_df['diff'] = (agg_df['Answer|mean'] - agg_df['GroundTruth|mean']).abs()
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "Answer|mean", "GroundTruth|mean", "diff"]].sort_values(by=['diff'], ascending=False)


,tweet,HITId,Answer|mean,GroundTruth|mean,diff
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,2.00,2.0,0.00
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,2.50,3.0,0.50
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,1.50,1.0,0.50
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,3.50,3.0,0.50
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,3.00,4.0,1.00
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,4.00,5.0,1.00
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,3.00,4.0,1.00
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,3.50,2.0,1.50
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,2.50,1.0,1.50
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,2.75,5.0,2.25


Which tasks had the highest STD DEV?

In [39]:
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "diff", "Answer|std"]].sort_values(by=['Answer|std'], ascending=False)


,tweet,HITId,diff,Answer|std
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,-0.50,2.121320
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,-2.25,2.061553
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,-1.00,1.732051
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,-1.00,1.414214
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,0.00,1.000000
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,1.50,0.707107
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,1.50,0.707107
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,0.50,0.707107
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,0.50,0.577350
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,-1.00,NaN
